In [9]:
%pip install FalkorDB python-dotenv yfinance


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import yfinance as yf
from falkordb import FalkorDB          # pip install FalkorDB
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# ───────────────────────────────────────────────
# 1. Connection Config
# ───────────────────────────────────────────────
HOST = "localhost"
PORT = 6379
PASSWORD = os.getenv("FALKORDB_PASSWORD")     # optional — most local instances have no password

# Connect (you can also add username=... if needed)
client = FalkorDB(
    host=HOST,
    port=PORT,
    password=PASSWORD if PASSWORD else "PASSWORD_NOT_SET"
)

# Select / create a named graph (like a database name)
GRAPH_NAME = "financial"                      # you can change this
graph = client.select_graph(GRAPH_NAME)

def ingest_financial_data(tickers):
    for symbol in tickers:
        # Fetch real data from Yahoo Finance
        stock = yf.Ticker(symbol)
        info = stock.info
        
        # Extract basic entities
        name = info.get('longName', symbol)
        sector = info.get('sector', 'Unknown')
        summary = info.get('longBusinessSummary', 'No summary available.')

        # Create Nodes and Relationships using Cypher
        # (MERGE is supported and behaves very similarly to Neo4j)
        query = """
        MERGE (c:Company {ticker: $ticker})
        SET c.name = $name, c.summary = $summary
        MERGE (s:Sector {name: $sector})
        MERGE (c)-[:IN_SECTOR]->(s)
        """
        graph.query(
            query,
            {
                "ticker": symbol,
                "name":   name,
                "sector": sector,
                "summary": summary
            }
        )
        
        print(f"Ingested: {name} in {sector} sector.")

def graph_rag_traversal(target_sector):
    query = """
    MATCH (s:Sector {name: $sector})<-[:IN_SECTOR]-(c:Company)
    RETURN c.name AS company, c.summary AS context
    """
    result = graph.query(query, {"sector": target_sector})
    
    contexts = []
    for row in result.result_set:
        company = row[0]   # company name
        summary  = row[1]  # summary text
        contexts.append(f"Data for {company}: {summary[:200]}...")
    
    return contexts



In [15]:

# ───────────────────────────────────────────────
# Example usage
# ───────────────────────────────────────────────

tickers = ["AAPL", "MSFT", "GOOGL", "TSLA"]
ingest_financial_data(tickers)

# Test retrieval
sector1 = "Technology"
results = graph_rag_traversal(sector1)
print("\nRetrieved context:")
for line in results:
    print(line)

Ingested: Apple Inc. in Technology sector.
Ingested: Microsoft Corporation in Technology sector.
Ingested: Alphabet Inc. in Communication Services sector.
Ingested: Tesla, Inc. in Consumer Cyclical sector.

Retrieved context:
Data for Apple Inc.: Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal ...
Data for Microsoft Corporation: Microsoft Corporation develops and supports software, services, devices, and solutions worldwide. The Productivity and Business Processes segment offers Microsoft 365 commercial, enterprise mobility +...


In [13]:
# Debug: see exactly what sectors exist
sectors_result = graph.query("""
MATCH (s:Sector)
RETURN DISTINCT s.name AS sector_name
ORDER BY s.name
""")

print("Existing sectors in the graph:")
for row in sectors_result.result_set:
    # row is a list → column 0 = sector_name (because only one column returned)
    print(f"  - '{row[0]}'")

Existing sectors in the graph:
  - 'Communication Services'
  - 'Consumer Cyclical'
  - 'Technology'
